In [3]:
#!pip install mediapipe opencv-python

In [4]:
#pip install pyautogui

In [1]:
import mediapipe as mp 
import numpy as np 
import cv2
import uuid
import os
import pyautogui
import collections
import math 

In [2]:
#Optional: Set pyautogui to fail-safe, moving mouse to top-left stops the script if it goes haywire
#pyautogui.FAILSAFE = True

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
cap = cv2.VideoCapture(0)

history = collections.deque(maxlen=5)
prev_x, prev_y = 0, 0
dead_zone = 20

right_state = False
left_state = False

smoothed_x = None
smoothed_y = None

alpha = 0.2  # smoothing factor (0.1 = very smooth, 0.3 = more responsive)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Detections
        print(results)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
       
            #/////////////////////////////////////////#
            
                
            for hand_landmarks in results.multi_hand_landmarks:
                
                # Get coordinates for specific landmarks
                # Landmarks are normalized (0 to 1), need to scale them to actual screen size
                h, w, c = frame.shape

                # Extracting the screen size 
                screen_w, screen_h = pyautogui.size()
                
                # --- LANDMARKS IN CAMERA SPACE (IMPORTANT!) ---
                index_x  = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * w
                index_y  = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * h
            
                thumb_x  = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x * w
                thumb_y  = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y * h
            
                middle_x = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * w
                middle_y = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * h
            
                ring_x   = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].x * w
                ring_y   = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y * h
            
                # --- GESTURE DISTANCES (Camera space!) ---
                pinch_distance = math.hypot(index_x - thumb_x, index_y - thumb_y)
                right_click_distance = math.hypot(middle_x - index_x, middle_y - index_y)
                left_click_distance  = math.hypot(ring_x - thumb_x, ring_y - thumb_y)
                
                # Define a threshold for the "pinch" gesture (you may need to adjust this value)
                pinch_threshold = 50 # Example threshold in pixels

                # Define the threshold to consider a click... dont know how relevant this would be. Review the number
                click_threshold = 40
                left_click_threshold = 50

                right_click_engaged = right_click_distance < click_threshold
                left_click_engaged = left_click_distance < left_click_threshold
                

                cursor_control = pinch_distance < pinch_threshold
            
                if cursor_control:
                    # Cursor Sensitivity tune this
                    sensitivity = 1.4
                   
                    # Compute moving average
                    # --- EMA SMOOTHING ---
                    raw_x = index_x
                    raw_y = index_y
                    
                    if smoothed_x is None:
                        # Initialize EMA the first frame
                        smoothed_x = raw_x
                        smoothed_y = raw_y
                    else:
                        smoothed_x = alpha * raw_x + (1 - alpha) * smoothed_x
                        smoothed_y = alpha * raw_y + (1 - alpha) * smoothed_y

                    
                    # Apply dead zone
                    if abs(smoothed_x - prev_x) > dead_zone or abs(smoothed_y - prev_y) > dead_zone:
                        pyautogui.moveTo(smoothed_x * sensitivity, smoothed_y * sensitivity)
                        prev_x, prev_y = smoothed_x, smoothed_y

                if right_click_engaged and not right_state :
                    pyautogui.click( button = 'right')  
                    right_state = True 
                elif not right_click_engaged:
                    right_state = False

                if left_click_engaged and not left_state :
                    pyautogui.click()
                    left_state = True
                elif not left_click_engaged:
                    left_state = False

                         
            #/////////////////////////////////////////#
            
        
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

2025-12-06 20:31:16.506 python[70102:1468525] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1765071077.444312 1468525 gl_context.cc:369] GL version: 2.1 (2.1 INTEL-23.0.26), renderer: Intel(R) Iris(TM) Plus Graphics 645
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1765071077.478876 1468898 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765071077.529997 1468903 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

W0000 00:00:1765071081.799098 1468897 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti